<a href="https://colab.research.google.com/github/neurobit-ai/eyeForm/blob/main/update_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
assert files.upload()

Saving eyeForm.db.xlsx to eyeForm.db.xlsx


In [2]:
import pandas as pd
sheet = pd.read_excel('eyeForm.db.xlsx', sheet_name='三總', skiprows=2)
db_version = f"v{sheet.iloc[-1]['時間戳記']}".split()[0].replace('-', '')
df = sheet[['性別', '實歲', '右眼軸長', '左眼軸長', '右眼危險度', '左眼危險度']].dropna(subset=['性別', '實歲', '右眼軸長', '左眼軸長'])
df

,性別,實歲,右眼軸長,左眼軸長,右眼危險度,左眼危險度
0,男,6歲11月,25.46,25.13,3.0,3.0
1,男,7歲2月,25.48,25.20,3.0,3.0
2,男,7歲4月,25.56,25.26,3.0,3.0
3,女,7歲7月,23.66,23.50,1.0,0.0
5,女,8歲3月,24.0,23.77,1.0,1.0
...,...,...,...,...,...,...
3979,女,3歲10個月,21.48,21.51,NaN,NaN
3980,男,3歲5個月,22.62,22.73,NaN,NaN
3981,男,3歲6個月,22.63,22.68,NaN,NaN
3982,男,5歲6個月,22.52,22.74,NaN,NaN


In [3]:
od = df.drop(columns='左眼軸長')
os = df.drop(columns='右眼軸長')
od.columns = ['性別', '實歲', '軸長', '右眼危險度', '左眼危險度']
os.columns = ['性別', '實歲', '軸長', '右眼危險度', '左眼危險度']
data = pd.concat([od, os])
data

,性別,實歲,軸長,右眼危險度,左眼危險度
0,男,6歲11月,25.46,3.0,3.0
1,男,7歲2月,25.48,3.0,3.0
2,男,7歲4月,25.56,3.0,3.0
3,女,7歲7月,23.66,1.0,0.0
5,女,8歲3月,24.0,1.0,1.0
...,...,...,...,...,...
3979,女,3歲10個月,21.51,NaN,NaN
3980,男,3歲5個月,22.73,NaN,NaN
3981,男,3歲6個月,22.68,NaN,NaN
3982,男,5歲6個月,22.74,NaN,NaN


In [4]:
data = data.drop(data[data['軸長'] == '< 20'].index)
data = data.drop(data[data['軸長'] == '> 30'].index)
data

,性別,實歲,軸長,右眼危險度,左眼危險度
0,男,6歲11月,25.46,3.0,3.0
1,男,7歲2月,25.48,3.0,3.0
2,男,7歲4月,25.56,3.0,3.0
3,女,7歲7月,23.66,1.0,0.0
5,女,8歲3月,24.0,1.0,1.0
...,...,...,...,...,...
3979,女,3歲10個月,21.51,NaN,NaN
3980,男,3歲5個月,22.73,NaN,NaN
3981,男,3歲6個月,22.68,NaN,NaN
3982,男,5歲6個月,22.74,NaN,NaN


In [5]:
age = data['實歲'].str.extract('(\d+)歲(\d+)')
age

,0,1
0,6,11
1,7,2
2,7,4
3,7,7
5,8,3
...,...,...
3979,3,10
3980,3,5
3981,3,6
3982,5,6


In [6]:
data['實歲'] = age[0].astype(int) + age[1].astype(int) / 12
data['年歲'] = age[0].astype(int) + (age[1].astype(int) >= 6).astype(int) # round() could be wrong
data['軸長'] = data['軸長'].astype(float)
data

,性別,實歲,軸長,右眼危險度,左眼危險度,年歲
0,男,6.916667,25.46,3.0,3.0,7
1,男,7.166667,25.48,3.0,3.0,7
2,男,7.333333,25.56,3.0,3.0,7
3,女,7.583333,23.66,1.0,0.0,8
5,女,8.250000,24.00,1.0,1.0,8
...,...,...,...,...,...,...
3979,女,3.833333,21.51,NaN,NaN,4
3980,男,3.416667,22.73,NaN,NaN,3
3981,男,3.500000,22.68,NaN,NaN,4
3982,男,5.500000,22.74,NaN,NaN,6


In [7]:
import numpy as np
def percentile(q):
    def function(a):
        return np.percentile(a, q)
    function.__name__ = f'P{q}'
    return function

In [8]:
quartile = data.groupby(['性別', '年歲'])['軸長'].aggregate([percentile(25), percentile(75)])
IQR = quartile['P75'] - quartile['P25']
lower_fence = quartile['P25'] - IQR * 1.5
upper_fence = quartile['P75'] + IQR * 1.5

In [9]:
for i in quartile.index:
    data.loc[(data['性別'] == i[0]) & (data['年歲'] == i[1]), 'lower_fence'] = lower_fence[i]
    data.loc[(data['性別'] == i[0]) & (data['年歲'] == i[1]), 'upper_fence'] = upper_fence[i]
data

,性別,實歲,軸長,右眼危險度,左眼危險度,年歲,lower_fence,upper_fence
0,男,6.916667,25.46,3.0,3.0,7,20.47250,26.29250
1,男,7.166667,25.48,3.0,3.0,7,20.47250,26.29250
2,男,7.333333,25.56,3.0,3.0,7,20.47250,26.29250
3,女,7.583333,23.66,1.0,0.0,8,21.29125,25.60125
5,女,8.250000,24.00,1.0,1.0,8,21.29125,25.60125
...,...,...,...,...,...,...,...,...
3979,女,3.833333,21.51,NaN,NaN,4,19.90125,23.47125
3980,男,3.416667,22.73,NaN,NaN,3,20.24000,24.98000
3981,男,3.500000,22.68,NaN,NaN,4,20.29125,24.20125
3982,男,5.500000,22.74,NaN,NaN,6,20.07500,25.93500


In [10]:
data_without_outliers = data[(data['軸長'] > data['lower_fence']) & (data['軸長'] < data['upper_fence'])]
data_without_outliers

,性別,實歲,軸長,右眼危險度,左眼危險度,年歲,lower_fence,upper_fence
0,男,6.916667,25.46,3.0,3.0,7,20.47250,26.29250
1,男,7.166667,25.48,3.0,3.0,7,20.47250,26.29250
2,男,7.333333,25.56,3.0,3.0,7,20.47250,26.29250
3,女,7.583333,23.66,1.0,0.0,8,21.29125,25.60125
5,女,8.250000,24.00,1.0,1.0,8,21.29125,25.60125
...,...,...,...,...,...,...,...,...
3978,女,3.416667,21.76,NaN,NaN,3,19.73500,23.07500
3979,女,3.833333,21.51,NaN,NaN,4,19.90125,23.47125
3980,男,3.416667,22.73,NaN,NaN,3,20.24000,24.98000
3981,男,3.500000,22.68,NaN,NaN,4,20.29125,24.20125


In [11]:
data_without_outliers['年歲'].value_counts()

8     459
6     447
5     443
7     397
9     371
10    357
4     295
11    215
12    188
13    115
3     113
14     99
15     83
16     40
17     11
2       9
18      6
1       6
21      2
0       2
Name: 年歲, dtype: int64

In [12]:
stacked_area = data_without_outliers.groupby(['性別', '年歲'])['軸長'].aggregate([percentile(0), percentile(50), percentile(75), percentile(90), percentile(100)])
stacked_area

P0     P50      P75     P90   P100
性別 年歲                                       
女  0   20.50  20.520  20.5300  20.536  20.54
   1   20.55  20.690  20.7450  20.765  20.77
   2   20.86  20.905  20.9700  21.024  21.06
   3   20.26  21.310  21.7850  22.194  23.06
   4   20.35  21.720  22.0200  22.789  23.33
   5   19.73  22.130  22.6200  23.021  24.50
   6   19.64  22.180  22.9300  23.326  24.57
   7   21.03  22.980  23.4275  24.008  24.84
   8   21.35  23.450  23.8600  24.620  25.54
   9   21.51  23.680  24.0600  24.556  25.57
   10  21.58  23.770  24.4650  24.920  25.89
   11  21.49  23.910  24.8000  25.190  25.90
   12  22.51  24.320  24.7200  25.070  25.33
   13  22.81  24.480  25.0100  25.680  26.66
   14  22.92  24.855  25.6550  25.932  26.20
   15  22.39  25.010  25.4475  26.078  26.44
   16  23.29  24.600  25.3250  25.840  26.54
   17  23.63  25.290  25.5900  25.872  26.01
   18  22.42  23.700  25.1400  25.806  26.25
   21  24.69  24.915  25.0275  25.095  25.14
男  2   21.88  22.060  22.0800  22.296  22.44
   3   20.99  22.610  23.2450  23.825  24.29
   4   20.37  22.170  22.7000  23.134  24.14
   5   20.48  22.640  23.3050  23.890  24.96
   6   20.13  22.840  23.7400  24.344  25.53
   7   20.52  23.560  24.1100  24.628  26.14
   8   21.45  23.965  24.4300  25.019  25.76
   9   22.45  24.245  24.6550  24.990  25.90
   10  21.99  24.510  25.1175  25.490  26.28
   11  22.36  24.510  25.2550  25.932  26.77
   12  23.02  24.665  25.1725  25.826  26.93
   13  23.26  24.885  25.1600  25.530  26.44
   14  23.21  24.820  25.1900  25.840  26.56
   15  23.54  24.840  25.0800  25.398  26.23
   16  22.40  25.410  26.6525  27.197  27.86
   17  26.55  27.010  27.4675  27.715  27.88
   18  25.46  25.695  25.8125  25.883  25.93

In [13]:
def level(sex, age, length):
    if length < stacked_area.loc[sex].loc[age].P50:
        return 0
    elif length < stacked_area.loc[sex].loc[age].P75:
        return 1
    elif length < stacked_area.loc[sex].loc[age].P90:
        return 2
    else:
        return 3

In [14]:
df = df.drop(df[df['右眼軸長'] == '< 20'].index)
df = df.drop(df[df['右眼軸長'] == '> 30'].index)
df = df.drop(df[df['左眼軸長'] == '< 20'].index)
df = df.drop(df[df['左眼軸長'] == '> 30'].index)
df

,性別,實歲,右眼軸長,左眼軸長,右眼危險度,左眼危險度
0,男,6歲11月,25.46,25.13,3.0,3.0
1,男,7歲2月,25.48,25.20,3.0,3.0
2,男,7歲4月,25.56,25.26,3.0,3.0
3,女,7歲7月,23.66,23.50,1.0,0.0
5,女,8歲3月,24.0,23.77,1.0,1.0
...,...,...,...,...,...,...
3979,女,3歲10個月,21.48,21.51,NaN,NaN
3980,男,3歲5個月,22.62,22.73,NaN,NaN
3981,男,3歲6個月,22.63,22.68,NaN,NaN
3982,男,5歲6個月,22.52,22.74,NaN,NaN


In [15]:
age = df['實歲'].str.extract('(\d+)歲(\d+)')
age

,0,1
0,6,11
1,7,2
2,7,4
3,7,7
5,8,3
...,...,...
3979,3,10
3980,3,5
3981,3,6
3982,5,6


In [16]:
df['實歲'] = age[0].astype(int) + age[1].astype(int) / 12
df['年歲'] = age[0].astype(int) + (age[1].astype(int) >= 6).astype(int) # round() could be wrong
df['右眼軸長'] = df['右眼軸長'].astype(float)
df['左眼軸長'] = df['左眼軸長'].astype(float)
df

,性別,實歲,右眼軸長,左眼軸長,右眼危險度,左眼危險度,年歲
0,男,6.916667,25.46,25.13,3.0,3.0,7
1,男,7.166667,25.48,25.20,3.0,3.0,7
2,男,7.333333,25.56,25.26,3.0,3.0,7
3,女,7.583333,23.66,23.50,1.0,0.0,8
5,女,8.250000,24.00,23.77,1.0,1.0,8
...,...,...,...,...,...,...,...
3979,女,3.833333,21.48,21.51,NaN,NaN,4
3980,男,3.416667,22.62,22.73,NaN,NaN,3
3981,男,3.500000,22.63,22.68,NaN,NaN,4
3982,男,5.500000,22.52,22.74,NaN,NaN,6


In [17]:
for i in df.index:
    sex = df.loc[i]['性別']
    age = df.loc[i]['年歲']
    df.loc[i, '右眼危險度'] = level(sex, age, df.loc[i]['右眼軸長'])
    df.loc[i, '左眼危險度'] = level(sex, age, df.loc[i]['左眼軸長'])
df

,性別,實歲,右眼軸長,左眼軸長,右眼危險度,左眼危險度,年歲
0,男,6.916667,25.46,25.13,3.0,3.0,7
1,男,7.166667,25.48,25.20,3.0,3.0,7
2,男,7.333333,25.56,25.26,3.0,3.0,7
3,女,7.583333,23.66,23.50,1.0,1.0,8
5,女,8.250000,24.00,23.77,2.0,1.0,8
...,...,...,...,...,...,...,...
3979,女,3.833333,21.48,21.51,0.0,0.0,4
3980,男,3.416667,22.62,22.73,1.0,1.0,3
3981,男,3.500000,22.63,22.68,1.0,1.0,4
3982,男,5.500000,22.52,22.74,0.0,0.0,6


In [18]:
sheet['右眼危險度'] = ''
sheet['左眼危險度'] = ''
sheet.loc[df.index, '右眼危險度'] = df['右眼危險度'].astype(int)
sheet.loc[df.index, '左眼危險度'] = df['左眼危險度'].astype(int)

In [19]:
sheet.to_excel('update_db.xlsx')
files.download('update_db.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>